In [10]:
from pyspark.sql import SparkSession

# Path to the Microsoft SQL Server JDBC driver
jdbc_driver_path = "/Users/liangou/Library/Application Support/JetBrains/DataGrip2024.2/jdbc-drivers/SQL Server/12.2.0/com/microsoft/sqlserver/mssql-jdbc/12.2.0.jre8/mssql-jdbc-12.2.0.jre8.jar"

# Create a Spark session and include the JDBC driver
spark = SparkSession.builder \
    .appName("SQLServerConnection") \
    .config("spark.jars", jdbc_driver_path) \
    .getOrCreate()

# Database connection details (replace these with the actual values from DataGrip)
# db_url = "jdbc:sqlserver://172.31.39.89:1433;databaseName=DaVinci"
db_url = "jdbc:sqlserver://172.31.39.89:1433;databaseName=DaVinciPrototype;encrypt=false;trustServerCertificate=true"
db_properties = {
    "user": "DvReader",      # Replace with your SQL Server username
    "password": "4QO4gNts7F?TKBH3Wa<>_^]8",  # Replace with your SQL Server password
    "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}





In [11]:
# Load data from the SQL Server database (replace "dbo.Addresses" with your table)
df = spark.read.jdbc(url=db_url, table="dbo.Addresses", properties=db_properties)

# Show the data
df.show(5)

+---+--------------------+-----+-----+-----------+-----+--------+-----------+---------+---------+--------------------+--------------------+-----------+------------+-----------+--------------+--------------------+----------+------------+----------+-------------+---------------+
| Id|               Line1|Line2|Line3|       City|State|Postcode|CountryCode|      Lat|      Lng|         CreatedDate|        ModifiedDate|CreatedById|ModifiedById|DeletedDate|IsActiveRecord|            Location|UnitNumber|StreetNumber|StreetName|GooglePlaceId|GooglePlaceName|
+---+--------------------+-----+-----+-----------+-----+--------+-----------+---------+---------+--------------------+--------------------+-----------+------------+-----------+--------------+--------------------+----------+------------+----------+-------------+---------------+
|  1|Darwin Dermatolog...| NULL| NULL|       Tiwi|   NT|    0810|         AU|-12.36264|130.87386|2023-07-05 07:08:...|2024-09-10 09:14:...|          2|        7549|  

In [6]:
# from pyspark.sql import SparkSession

# # Path to the Microsoft SQL Server JDBC driver
# jdbc_driver_path = "/Users/liangou/Library/Application Support/JetBrains/DataGrip2024.2/jdbc-drivers/SQL Server/12.2.0/com/microsoft/sqlserver/mssql-jdbc/12.2.0.jre8/mssql-jdbc-12.2.0.jre8.jar"

# # Create a Spark session and include the JDBC driver
# spark = SparkSession.builder \
#     .appName("SQLServerConnection") \
#     .config("spark.jars", jdbc_driver_path) \
#     .getOrCreate()

# # Database connection details (replace these with the actual values from DataGrip)
# # db_url = "jdbc:sqlserver://172.31.39.89:1433;databaseName=DaVinci"
# db_url = "jdbc:sqlserver://172.31.39.89:1433;databaseName=DaVinciPrototype;encrypt=false;trustServerCertificate=true"
# db_properties = {
#     "user": "DvReader",      # Replace with your SQL Server username
#     "password": "4QO4gNts7F?TKBH3Wa<>_^]8",  # Replace with your SQL Server password
#     "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
# }



In [12]:
column_query = "(SELECT TABLE_SCHEMA, TABLE_NAME, COLUMN_NAME, DATA_TYPE FROM INFORMATION_SCHEMA.COLUMNS) AS columns_list"
columns_df = spark.read.jdbc(url=db_url, table=column_query, properties=db_properties)
columns_df.show(truncate=False)

+------------+-----------------+---------------------+---------+
|TABLE_SCHEMA|TABLE_NAME       |COLUMN_NAME          |DATA_TYPE|
+------------+-----------------+---------------------+---------+
|dbo         |ReferralDocuments|Id                   |bigint   |
|dbo         |ReferralDocuments|SpecialistId         |bigint   |
|dbo         |ReferralDocuments|Notes                |nvarchar |
|dbo         |ReferralDocuments|ConsultationId       |bigint   |
|dbo         |ReferralDocuments|FileId               |bigint   |
|dbo         |ReferralDocuments|CreatedDate          |datetime |
|dbo         |ReferralDocuments|ModifiedDate         |datetime |
|dbo         |ReferralDocuments|CreatedById          |bigint   |
|dbo         |ReferralDocuments|ModifiedById         |bigint   |
|dbo         |ReferralDocuments|DeletedDate          |datetime |
|dbo         |ReferralDocuments|IsActiveRecord       |bit      |
|dbo         |ReferralDocuments|SpecialistName       |nvarchar |
|dbo         |ReferralDoc

In [24]:
import pyodbc
import pandas as pd
from pyspark.sql import SparkSession

# Initialize the Spark session
spark = SparkSession.builder \
    .appName("SQLServerConnection") \
    .getOrCreate()

# Database connection details
db_url = "jdbc:sqlserver://172.31.39.89:1433;databaseName=DaVinciPrototype;encrypt=false;trustServerCertificate=true"
username = "DvReader"
password = "4QO4gNts7F?TKBH3Wa<>_^]8"

# Connect using pyodbc
conn_str = (
    'DRIVER={ODBC Driver 17 for SQL Server};'
    f'SERVER=172.31.39.89,1433;DATABASE=DaVinciPrototype;'
    f'UID={username};PWD={password};Encrypt=no;'
)
conn = pyodbc.connect(conn_str)

# SQL query
sql_query = """
SELECT
    pp.Gender,
    pp.DateOfBirth,
    pp.Allergies,
    cr.ConditionDescription,
    cr.ConsultationType,
    ct.TranscriptionText,
    ct.ConsultationId,
    ct.CreatedDate
FROM
    PatientProfiles pp
JOIN
    ConsultationRequests cr ON pp.Id = cr.PatientProfileId
JOIN
    Consultations c ON cr.Id = c.ConsultationRequestId
LEFT JOIN
    ConsultationCallTranscriptions ct ON c.Id = ct.ConsultationId
WHERE
    ct.TranscriptionText IS NOT NULL
ORDER BY
    pp.LastName, c.StartDate
"""

# Fetch data using pandas
df = pd.read_sql(sql_query, conn)

# Convert pandas DataFrame to Spark DataFrame
spark_df = spark.createDataFrame(df)

# Show the results
spark_df.show(truncate=False)

# Close the connection
conn.close()

/var/folders/ft/0w1py1ps44b_2sp_9vws3kgc0000gn/T/ipykernel_11712/2207421976.py:49: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_query, conn)
24/09/23 15:01:59 WARN TaskSetManager: Stage 9 contains a task of very large size (29658 KiB). The maximum recommended task size is 1000 KiB.


+------+-----------+---------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

24/09/23 15:02:03 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 9 (TID 9): Attempting to kill Python Worker


In [25]:
# Write to Delta Lake - Patient Table
consulation_table="./consulation_table"
spark_df.write.format("delta").mode("overwrite").save(consulation_table)

# Write to Delta Lake - Consultation Table
spark_df.write.format("delta").mode("overwrite").save(consulation_table)

# Close the connection
conn.close()

# Stop the Spark session
spark.stop()

Py4JJavaError: An error occurred while calling o190.save.
: org.apache.spark.SparkClassNotFoundException: [DATA_SOURCE_NOT_FOUND] Failed to find the data source: delta. Please find packages at `https://spark.apache.org/third-party-projects.html`.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.dataSourceNotFoundError(QueryExecutionErrors.scala:725)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:647)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSourceV2(DataSource.scala:697)
	at org.apache.spark.sql.DataFrameWriter.lookupV2Provider(DataFrameWriter.scala:863)
	at org.apache.spark.sql.DataFrameWriter.saveInternal(DataFrameWriter.scala:257)
	at org.apache.spark.sql.DataFrameWriter.save(DataFrameWriter.scala:240)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at java.base/jdk.inte

24/09/23 17:46:47 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1541411 ms exceeds timeout 120000 ms
24/09/23 17:46:47 WARN SparkContext: Killing executors is not supported by current scheduler.
24/09/23 17:46:51 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$